# Afiniczne Ciągłe Automaty Komórkowe (ACCAs)

Wykład: https://github.com/houp/ca-class/blob/main/slides/lecture7.pdf

## Importy

In [1]:
import math
import random
import sympy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy import symbols, Poly
from joblib import Parallel, delayed

## zad.13
Napisz program, który dla zadanego LUT (binarnego) oraz cLUT (real-valued) wygeneruje wielomian odpowiadający CA / ACCA.

Wersja dla normalnych ludzi: zakładamy, że promień sąsiedztwa = 1.

Wersja z (*): piszemy ogólny program dla dowolnego promienia sąsiedztwa.

Tak czy inaczej do napisania jest funkcja:\
**get_ca_polynomial(lut)** -> polynomial

Użyj SymPy lub podobnej biliboteki do operacji na wielomianach.

In [2]:
# tak powinno wyglądać dla reguły 232
x, y, z = symbols('x y z')
polynomial = 1*x*y*z + 1*x*y*(1-z) + 1*x*(1-y)*z + 0*x*(1-y)*(1-z) + 1*(1-x)*y*z + 0*(1-x)*y*(1-z) + 0*(1-x)*y*(1-z) + 0*(1-x)*(1-y)*(1-z)
polynomial

x*y*z + x*y*(1 - z) + x*z*(1 - y) + y*z*(1 - x)

In [3]:
def get_ca_polynomial(lut: np.ndarray) -> Poly:
    x, y, z = symbols('x y z')
    polynomial = (lut[0]*x*y*z + lut[1]*x*y*(1-z) + lut[2]*x*(1-y)*z + lut[3]*x*(1-y)*(1-z) +
                  lut[4]*(1-x)*y*z + lut[5]*(1-x)*y*(1-z) + lut[6]*(1-x)*(1-y)*z + lut[7]*(1-x)*(1-y)*(1-z))
    return polynomial


In [4]:
def eca_get_lut(rule_num: int) -> np.ndarray:
    return np.array([int(x) for x in bin(rule_num)[2:].zfill(8)], dtype=np.uint8)


In [5]:
lut = eca_get_lut(232)
get_ca_polynomial(lut)

x*y*z + x*y*(1 - z) + x*z*(1 - y) + y*z*(1 - x)

## zad.14
Korzystając z rozwiązania Zadania 13 oraz z wcześniejszych programów, napisz funkcję, która dla zadanego cLUT i zadanej konfiguracji początkowej generuje space-time diagram ACCA.

**acca_space_time(clut, initial_conf, step)** -> space time diagram w matplotlib

In [ ]:
# traffic = wolfram_rule_to_lut(184)
# majority = wolfram_rule_to_lut(232)

# alpha=0.9
# traffic_majority_clut = alpha * traffic + (1-alpha) * majority
# conf = np.random.randint(0,2,49)
# acca_space_time(traffic_majority_clut, conf, 49)

In [ ]:
# wziąć wielomian i go do czegoś użyć, do następnych stanów automatów komórkowych